#### 测试，数值型特征

In [1]:
from math import exp
import numpy as np
from random import normalvariate  # 正态分布
from datetime import datetime
import pandas as pd

In [2]:
trainData = 'C:/Users/hedy/Downloads/fm/diabetes_train.txt'   #请换为自己文件的路径
testData = 'C:/Users/hedy/Downloads/fm/diabetes_test.txt'

In [3]:
def preprocessData(data):
    """预处理"""
    # 获取特征
    feature = np.array(data.iloc[:, :-1])
    # 将标签转为 1 和 -1
    label = data.iloc[:, -1].map(lambda x: 1 if x == 1 else -1)
    
    # 将数组按行进行归一化（最大最小值归一化）
    zmax, zmin = feature.max(axis=0), feature.min(axis=0)
    feature = (feature - zmin) / (zmax - zmin)
    
    label = np.array(label)
    
    return feature, label

In [4]:
def sigmoid(inx):
    #return 1. / (1. + exp(-max(min(inx, 15.), -15.)))
    return 1.0 / (1 + exp(-inx))

In [5]:
np.shape

<function numpy.shape>

In [6]:
def SGD_FM(dataMatrix, classLabels, k, iter):
    '''
    :param dataMatrix:  特征矩阵 mat
    :param classLabels: 类别矩阵 列表
    :param k:           辅助向量的大小
    :param iter:        迭代次数
    :return:
    '''
    m, n = np.shape(dataMatrix)   #矩阵的行列数，即样本数和特征数
    
    alpha = 0.01
    # 初始化参数
    # w = random.randn(n, 1)#其中n是特征的个数
    w = np.zeros((n, 1))      #一阶特征的系数
    w_0 = 0.
    v = normalvariate(0, 0.2) * np.ones((n, k))   #即生成辅助向量，用来训练二阶交叉特征的系数
    
    for it in range(iter):
        for x in range(m):  # 随机优化，每次只使用一个样本
            # 二阶项的计算
            inter_1 = dataMatrix[x] * v
            inter_2 = np.multiply(dataMatrix[x], dataMatrix[x]) * np.multiply(v, v)  #二阶交叉项的计算
            interaction = sum(np.multiply(inter_1, inter_1) - inter_2) / 2.       #二阶交叉项计算完成
            
            p = w_0 + dataMatrix[x] * w + interaction  # 计算预测的输出，即FM的全部项之和
            loss = 1-sigmoid(classLabels[x] * p[0, 0])    #计算损失
            
            w_0 = w_0 + alpha * loss * classLabels[x]
            
            for i in range(n):
                if dataMatrix[x, i] != 0:
                    w[i, 0] = w[i, 0] + alpha * loss * classLabels[x] * dataMatrix[x, i]
                    for j in range(k):
                        v[i, j] = v[i, j] + alpha * loss * classLabels[x] * (dataMatrix[x, i] * inter_1[0, j] - v[i, j] * dataMatrix[x, i] * dataMatrix[x, i])
        print("第{}次迭代后的损失为{}".format(it, loss))
        
    return w_0, w, v

In [7]:
def getAccuracy(dataMatrix, classLabels, w_0, w, v):
    m, n = shape(dataMatrix)
    allItem = 0
    error = 0
    result = []
    for x in range(m):   #计算每一个样本的误差
        allItem += 1
        inter_1 = dataMatrix[x] * v
        inter_2 = np.multiply(dataMatrix[x], dataMatrix[x]) * np.multiply(v, v)
        interaction = sum(np.multiply(inter_1, inter_1) - inter_2) / 2.
        p = w_0 + dataMatrix[x] * w + interaction  # 计算预测的输出
        
        pre = sigmoid(p[0, 0])
        result.append(pre)
        
        if pre < 0.5 and classLabels[x] == 1.0:
            error += 1
        elif pre >= 0.5 and classLabels[x] == -1.0:
            error += 1
        else:
            continue
            
    return float(error) / allItem

In [8]:
if __name__ == '__main__':
    train = pd.read_csv(trainData)
    test = pd.read_csv(testData)
    dataTrain, labelTrain = preprocessData(train)
    dataTest, labelTest = preprocessData(test)
    date_startTrain = datetime.now()
    
    print    ("开始训练")
    
    w_0, w, v = SGD_FM(np.mat(dataTrain), labelTrain, 20, 200)
    print("训练准确性为：%f" % (1 - getAccuracy(mat(dataTrain), labelTrain, w_0, w, v)))
    
    date_endTrain = datetime.now()
    print("训练用时为：%s" % (date_endTrain - date_startTrain))
    
    print("开始测试")
    print("测试准确性为：%f" % (1 - getAccuracy(np.mat(dataTest), labelTest, w_0, w, v)))

开始训练
第0次迭代后的损失为0.3911898262308078
第1次迭代后的损失为0.40130266249842506
第2次迭代后的损失为0.42037798212389
第3次迭代后的损失为0.43813110118777243
第4次迭代后的损失为0.4534829893947645
第5次迭代后的损失为0.46629410857294096
第6次迭代后的损失为0.47672997630527836
第7次迭代后的损失为0.485067441395417
第8次迭代后的损失为0.49160565591446825
第9次迭代后的损失为0.49662664083034713
第10次迭代后的损失为0.50037954830209
第11次迭代后的损失为0.5030767154627553
第12次迭代后的损失为0.5048955334590437
第13次迭代后的损失为0.505982834736846
第14次迭代后的损失为0.5064599655083015
第15次迭代后的损失为0.5064275945885104
第16次迭代后的损失为0.5059698437046845
第17次迭代后的损失为0.5051576301825677
第18次迭代后的损失为0.5040512720462818
第19次迭代后的损失为0.502702474234043
第20次迭代后的损失为0.5011558308893636
第21次迭代后的损失为0.4994499675624756
第22次迭代后的损失为0.4976184244411661
第23次迭代后的损失为0.49569035674170214
第24次迭代后的损失为0.493691106018311
第25次迭代后的损失为0.49164267840181786
第26次迭代后的损失为0.4895641529350907
第27次迭代后的损失为0.4874720346134047
第28次迭代后的损失为0.48538056148653563
第29次迭代后的损失为0.48330197223692417
第30次迭代后的损失为0.48124673916032445
第31次迭代后的损失为0.4792237707957535
第32次迭代后的损失为0.4772405881408075
第33次迭代后的损失为0

NameError: name 'mat' is not defined